In [14]:
class Symbol:
    def __init__(self, symbol, address):
        self.symbol = symbol
        self.address = address

class Opcode:
    def __init__(self, mnemonic, machine_code):
        self.mnemonic = mnemonic
        self.machine_code = machine_code

optab = [
    Opcode("MOV", 0x10), Opcode("ADD", 0x20), Opcode("SUB", 0x30),
    Opcode("JMP", 0x40), Opcode("INP", 0x50), Opcode("STO", 0x60),
    Opcode("OUT", 0x70), Opcode("HLT", 0x80), Opcode("DS", 0x90),
    Opcode("START", -1)
]

symtab = []
instructions = []
current_location = 1000

def search_optab(mnemonic):

    return next((opt.machine_code for opt in optab if opt.mnemonic == mnemonic), -1)

def add_symbol(symbol, address):

    if not any(sym.symbol == symbol for sym in symtab):
        symtab.append(Symbol(symbol, address))

def get_symbol_address(symbol):

    for sym in symtab:
        if sym.symbol == symbol:
            return sym.address
    return None

def parse_instruction(line):

    global current_location
    parts = line.split()

    if not parts:
        return

    mnemonic = parts[0]
    operand = parts[1] if len(parts) > 1 else ""

    if mnemonic == "START" and operand:
        current_location = int(operand)
        print(f"Starting address set to {current_location}")
        return

    machine_code = search_optab(mnemonic)
    if machine_code == -1:
        print(f"Error: Invalid mnemonic '{mnemonic}'")
        return

    if operand and mnemonic not in ["DS", "START"]:
        add_symbol(operand, current_location + 1)

    instructions.append((mnemonic, operand))

    print(f"Location {current_location}: {mnemonic} {operand} --> Machine Code: 0x{machine_code:X}")
    current_location += 4

def main():
    print("Enter assembly code (type 'HLT' to finish):")
    while True:
        line = input().strip()
        if line == "HLT":
            break
        parse_instruction(line)

    print("\nSymbol Table:")
    if symtab:
        for sym in symtab:
            print(f"{sym.symbol}\t{sym.address}")
    else:
        print("No symbols defined.")

    print("\nOpcode Table:")
    for opt in optab:
        if opt.mnemonic == "START":
            continue

        operand_addresses = [
            get_symbol_address(operand)
            for mnemonic, operand in instructions
            if mnemonic == opt.mnemonic and operand
        ]

        if operand_addresses:
            for address in operand_addresses:
                if address is not None:
                    print(f"{opt.mnemonic}\t{address}")
        else:
            print(f"{opt.mnemonic}\tNo operand")

if __name__ == "__main__":
    main()


Enter assembly code (type 'HLT' to finish):
INP A
Location 1000: INP A --> Machine Code: 0x50
INP B
Location 1004: INP B --> Machine Code: 0x50
ADD B
Location 1008: ADD B --> Machine Code: 0x20
OUT C
Location 1012: OUT C --> Machine Code: 0x70
STO C
Location 1016: STO C --> Machine Code: 0x60
HLT

Symbol Table:
A	1001
B	1005
C	1013

Opcode Table:
MOV	No operand
ADD	1005
SUB	No operand
JMP	No operand
INP	1001
INP	1005
STO	1013
OUT	1013
HLT	No operand
DS	No operand


In [15]:
class Symbol:
    def __init__(self, symbol, address):
        self.symbol, self.address = symbol, address

class Opcode:
    def __init__(self, mnemonic, machine_code):
        self.mnemonic, self.machine_code = mnemonic, machine_code

optab = {op.mnemonic: op.machine_code for op in [
    Opcode("MOV", 0x10), Opcode("ADD", 0x20), Opcode("SUB", 0x30),
    Opcode("JMP", 0x40), Opcode("INP", 0x50), Opcode("STO", 0x60),
    Opcode("OUT", 0x70), Opcode("HLT", 0x80), Opcode("DS", 0x90), Opcode("START", -1)]}

symtab, instructions, current_location = {}, [], 1000

def parse_instruction(line):
    global current_location
    parts = line.split()
    if not parts: return
    mnemonic, operand = parts[0], parts[1] if len(parts) > 1 else ""
    if mnemonic == "START" and operand:
        current_location = int(operand)
        return
    if mnemonic not in optab:
        print(f"Error: Invalid mnemonic '{mnemonic}'")
        return
    if operand and mnemonic not in ["DS", "START"]:
        symtab.setdefault(operand, current_location + 1)
    instructions.append((mnemonic, operand))
    print(f"Location {current_location}: {mnemonic} {operand} --> Machine Code: 0x{optab[mnemonic]:X}")
    current_location += 4

def main():
    print("Enter assembly code (type 'HLT' to finish):")
    while (line := input().strip()) != "HLT":
        parse_instruction(line)

    print("\nSymbol Table:")
    print("\n".join(f"{sym}\t{addr}" for sym, addr in symtab.items()) if symtab else "No symbols defined.")

    print("\nOpcode Table:")
    for mnemonic in optab:
        if mnemonic != "START":
            addresses = [symtab.get(operand, "No operand") for op, operand in instructions if op == mnemonic]
            print(f"{mnemonic}\t" + ("\n".join(map(str, addresses)) if addresses else "No operand"))

if __name__ == "__main__":
    main()


Enter assembly code (type 'HLT' to finish):
INP A
Location 1000: INP A --> Machine Code: 0x50
INP B
Location 1004: INP B --> Machine Code: 0x50
ADD B
Location 1008: ADD B --> Machine Code: 0x20
OUT C
Location 1012: OUT C --> Machine Code: 0x70
STO C
Location 1016: STO C --> Machine Code: 0x60
HLT

Symbol Table:
A	1001
B	1005
C	1013

Opcode Table:
MOV	No operand
ADD	1005
SUB	No operand
JMP	No operand
INP	1001
1005
STO	1013
OUT	1013
HLT	No operand
DS	No operand
